## Test of different regression models

In [2]:
import numpy as np
import os
import pandas as pd
from sklearn.pipeline import make_pipeline
import itertools
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import joblib
from scipy.stats import pearsonr

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from tabpfn import TabPFNRegressor  
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
import xgboost as xgb
from catboost import CatBoostRegressor

ModuleNotFoundError: No module named 'tabpfn'

In [6]:
import pandas as pd

In [3]:
# please specify the path to the root directory of the repository
ROOT_PATH = '/mnt/processed/wolffjoa/hyperparameter_publication/Revision_1/'

In [4]:
output_path = os.path.join(ROOT_PATH, 'regression_model_results_revision')

In [5]:
os.makedirs(output_path, exist_ok=True)

In [12]:
merged_df = pd.read_csv(os.path.join(ROOT_PATH, 'merged_result_hicgan.txt'), sep='\t')

In [14]:
merged_df = merged_df.dropna()

In [15]:
merged_df

,relevant,ENT3C,QuASAR-Rep,HiC-Spector,GenomeDISCO,Pearson,TAD Score MSE,TAD Fraction,TAD Fraction Exact Match,HiCRep,ELO
0,8c4d9b9a,0.708279,0.597979,0.248,0.403,0.276921,1.637423,0.654424,0.043406,0.343147,-0.32
2,c3fb3f94,0.395600,0.727716,0.392,0.512,0.193879,1.561542,0.741235,0.033389,0.237585,9.55
3,9534066a,0.397781,0.702144,0.418,0.239,0.134168,1.724281,0.742905,0.061770,0.186757,-11.24
4,173a3725,0.499141,0.705062,0.415,0.522,0.135822,1.766373,0.415693,0.023372,0.176175,-11.36
5,6c97462d,0.353742,0.685396,0.359,0.404,0.057183,1.627826,0.851419,0.048414,0.037917,4.01
...,...,...,...,...,...,...,...,...,...,...,...
419,dbec84ea,0.770611,0.709142,0.223,0.589,0.365175,1.620448,0.774624,0.060100,0.437561,-0.93
420,a28f0083,0.754363,0.717140,0.376,0.473,0.131805,1.498451,0.268781,0.008347,0.175941,-14.18
421,08d06603,0.351834,0.650648,0.379,0.408,-0.088131,1.648474,0.580968,0.050083,-0.119623,-1.65
422,980b57d9,0.936739,0.794216,0.446,0.641,0.332167,1.605357,1.240401,0.141903,0.429618,12.16


In [10]:
# Prepare the data
features = [col for col in merged_df.columns if col not in ['ELO', 'relevant']]
# Split the data into training and testing sets
X = merged_df[features]
y = merged_df['ELO']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a dictionary to store the models
models = {}

# Generate all combinations of 2 to all features
combinations = []
for r in range(2, len(features) + 1):
    combinations.extend(itertools.combinations(features, r))


[('ENT3C', 'QuASAR-Rep'),
 ('ENT3C', 'HiC-Spector'),
 ('ENT3C', 'GenomeDISCO'),
 ('ENT3C', 'Pearson'),
 ('ENT3C', 'TAD Score MSE'),
 ('ENT3C', 'TAD Fraction'),
 ('ENT3C', 'TAD Fraction Exact Match'),
 ('ENT3C', 'HiCRep'),
 ('QuASAR-Rep', 'HiC-Spector'),
 ('QuASAR-Rep', 'GenomeDISCO'),
 ('QuASAR-Rep', 'Pearson'),
 ('QuASAR-Rep', 'TAD Score MSE'),
 ('QuASAR-Rep', 'TAD Fraction'),
 ('QuASAR-Rep', 'TAD Fraction Exact Match'),
 ('QuASAR-Rep', 'HiCRep'),
 ('HiC-Spector', 'GenomeDISCO'),
 ('HiC-Spector', 'Pearson'),
 ('HiC-Spector', 'TAD Score MSE'),
 ('HiC-Spector', 'TAD Fraction'),
 ('HiC-Spector', 'TAD Fraction Exact Match'),
 ('HiC-Spector', 'HiCRep'),
 ('GenomeDISCO', 'Pearson'),
 ('GenomeDISCO', 'TAD Score MSE'),
 ('GenomeDISCO', 'TAD Fraction'),
 ('GenomeDISCO', 'TAD Fraction Exact Match'),
 ('GenomeDISCO', 'HiCRep'),
 ('Pearson', 'TAD Score MSE'),
 ('Pearson', 'TAD Fraction'),
 ('Pearson', 'TAD Fraction Exact Match'),
 ('Pearson', 'HiCRep'),
 ('TAD Score MSE', 'TAD Fraction'),
 ('TAD 

In [11]:
len(combinations)

502

In [ ]:
# Prepare the data
features = [col for col in merged_df.columns if col not in ['ELO', 'relevant']]
# Split the data into training and testing sets
X = merged_df[features]
y = merged_df['ELO']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a dictionary to store the models
models = {}

# Generate all combinations of 2 to all features
combinations = []
for r in range(2, len(features) + 1):
    combinations.extend(itertools.combinations(features, r))



# Initialize a dictionary to store the results
results = {}

# Save the MSE, Pearson correlation, and p-value in the dictionary

# Perform regression for each combination and degree
algorithms = ['xgboost', 'catboost', 'TabPFN', 'linear regression', 'random forest', 'gradientBoosting']

for algo in algorithms:
    print(f"Running {algo} regression...")
    for i, combo in enumerate(combinations):
        mse_scores = []
        degrees = range(1, 9)
        for degree in degrees:
            poly = PolynomialFeatures(degree)
            X_poly_train = poly.fit_transform(X_train[list(combo)])
            X_poly_test = poly.transform(X_test[list(combo)])

            if algo == 'linear regression':
                model = LinearRegression()
            elif algo == 'random forest':
                model = RandomForestRegressor(random_state=42, n_jobs=16)
            elif algo == 'gradientBoosting':
                model = GradientBoostingRegressor(random_state=42)
            elif algo == 'xgboost':
                model = xgb.XGBRegressor(random_state=42, verbosity=0, device='cuda')
            elif algo == 'catboost':
                model = CatBoostRegressor(random_state=42, verbose=0, task_type="GPU", devices='0')
            elif algo == 'TabPFN':
                model = TabPFNRegressor(device="cuda")

            model.fit(X_poly_train, y_train)
            mse = -cross_val_score(model, X_poly_test, y_test, cv=5, scoring='neg_mean_squared_error').mean()
            mse_scores.append(mse)

        best_mse = min(mse_scores)
        best_degree = list(degrees)[mse_scores.index(best_mse)]

        if algo == 'linear regression':
            best_model = make_pipeline(PolynomialFeatures(best_degree), LinearRegression())
        elif algo == 'random forest':
            best_model = make_pipeline(PolynomialFeatures(best_degree), RandomForestRegressor(random_state=42))
        elif algo == 'gradientBoosting':
            best_model = make_pipeline(PolynomialFeatures(best_degree), GradientBoostingRegressor(random_state=42))
        elif algo == 'xgboost':
            best_model = make_pipeline(PolynomialFeatures(best_degree), xgb.XGBRegressor(random_state=42, verbosity=0))
        elif algo == 'catboost':
            best_model = make_pipeline(PolynomialFeatures(best_degree), CatBoostRegressor(random_state=42, verbose=0))
        elif algo == 'TabPFN':
            best_model = make_pipeline(PolynomialFeatures(best_degree), TabPFNRegressor())

        
        best_model.fit(X_train[list(combo)], y_train)
        combo_str = " ".join([feat for feat in combo])
        key = f"{algo}_{combo}_degree_{best_degree}"
        models[key] = best_model
        joblib.dump(best_model, os.path.join(output_path, f"best_model_{algo}_{combo_str}_degree_{best_degree}.pkl"))

        y_pred = best_model.predict(X_test[list(combo)])
        test_mse = mean_squared_error(y_test, y_pred)
        pearson_corr, p_value = pearsonr(y_test, y_pred)
        print(f"{algo} | {combo_str} | Degree {best_degree} | Test MSE: {test_mse:.2e} | Pearson r: {pearson_corr:.2f} | p-value: {p_value:.2e}")

        results[f"{algo}_{combo_str}"] = {
            'best_degree': best_degree,
            'test_mse': test_mse,
            'pearson_corr': pearson_corr,
            'p_value': p_value
        }



Running xgboost regression...
xgboost | ENT3C QuASAR-Rep | Degree 2 | Test MSE: 7.65e+01 | Pearson r: 0.22 | p-value: 6.39e-02
xgboost | ENT3C HiC-Spector | Degree 5 | Test MSE: 9.33e+01 | Pearson r: 0.02 | p-value: 8.71e-01
xgboost | ENT3C GenomeDISCO | Degree 6 | Test MSE: 9.19e+01 | Pearson r: 0.08 | p-value: 4.97e-01
xgboost | ENT3C Pearson | Degree 8 | Test MSE: 9.03e+01 | Pearson r: 0.15 | p-value: 2.13e-01
xgboost | ENT3C TAD Score MSE | Degree 1 | Test MSE: 8.01e+01 | Pearson r: 0.32 | p-value: 6.24e-03
xgboost | ENT3C TAD Fraction | Degree 8 | Test MSE: 2.57e+01 | Pearson r: 0.78 | p-value: 5.34e-16
xgboost | ENT3C TAD Fraction Exact Match | Degree 8 | Test MSE: 4.36e+01 | Pearson r: 0.58 | p-value: 5.59e-08
xgboost | ENT3C HiCRep | Degree 8 | Test MSE: 8.58e+01 | Pearson r: 0.19 | p-value: 1.14e-01
xgboost | QuASAR-Rep HiC-Spector | Degree 4 | Test MSE: 6.88e+01 | Pearson r: 0.37 | p-value: 1.38e-03
xgboost | QuASAR-Rep GenomeDISCO | Degree 2 | Test MSE: 8.66e+01 | Pearson r:

ValueError: Number of features 792 in the input data is greater than the maximum number of features 500 officially supported by the TabPFN model. Set `ignore_pretraining_limits=True` to override this error!

In [ ]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.reset_index(inplace=True)
results_df.rename(columns={'index': 'model_combo'}, inplace=True)
results_df
results_df.to_csv(os.path.join(output_path, "results_xgboost_cat_tab_df.csv"), index=False)
